<div style="border-radius: 10px; box-shadow: 0px 0px 2px; border: 1px solid; background:#eaeaea; position: relative; padding: 10px; padding-left: 10px;">
<h7 style="color:black; margin-bottom:20px"> 

# <span style="color:#143a51">Анализ трансляций НМИЦ онкологии им. Н. Н. Петрова <span> 
    
---
---   
    
**Заказчик:**    
представитель НМИЦ онкологии им. Н. Н. Петрова

**Цель:** 
необходимо обработать данные по посещаемости онкофорумов и вебинаров, исследовать данные, определить наиболее интересные темы, популярных спикеров, создать дашборд и предоставить рекомендации для будущих мероприятий.
    
---     
    
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
- [данные о зрителях](#1)
- [данные с расписанием](#2)
- [объединение таблиц](#3)
3. [Исследовательский анализ данных](#eda)
4. [Дашборд](#dash)
5. [Cписок зрителей для рекомендаций](#recommendation)
6. [Вывод](#finish)
---

## Открытие данных
<a id="start"></a>

In [1]:
# импортируем библиотеки 
import pandas as pd
import numpy as np
import re
from datetime import datetime,  date, time, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly_calplot import calplot
import dash
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output, State
from dash import Dash, dash_table
import plotly.graph_objects as go
import textwrap
from textwrap import TextWrapper, dedent
import warnings
warnings.filterwarnings('ignore')
# устанавливаем параметры отображение таблицы на экране 
pd.options.display.float_format ='{:.2f}'.format
pd.options.display.max_columns = 40 
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# считаем данные и сохраним в переменные
df = pd.read_csv('Все мероприятия NDA.csv', sep=";")
df_shedule = pd.read_csv('Расписание мероприятий NDA.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Все мероприятия NDA.csv'

In [ ]:
# создадим функцию df_info, которая будет выводить основную информацию по датафрейму
def df_info(data):
    display(data.head())
    display(data.info())
    print('Пропуски:',  data.isna().sum(), sep='\n')
    print()
    print('Дубликаты:', data.duplicated().sum())
    print()
    # визуализируем пропущенные значения в данных с помощью тепловой карты
    sns.set(rc = {'figure.figsize': (10, 6)})
    sns.heatmap(data.isna(), cbar=False).set_title('Распределение  пропусков', fontsize='16');

### Данные с событиями
<a id="1"></a>

In [ ]:
df_info(df)

`Для начала необходимо:`
- удалить пустые столбцы,
- изменить название столбцов,
- удалить пустые строки, 
- удалить дубликаты,
- посмотреть на пропуски в столбце id зрителей


In [ ]:
# удалим пустые строки
df.drop(['IP', 'Unnamed: 22'], axis=1, inplace=True)
# приведем название столбцов к snake_case
df.columns = df.columns.str.strip().str.lower()
df = df.rename(columns={'uid' : 'user_id',
                        'поток' : 'potok',
                        'устройство' : 'device',
                        'оп.сис' : 'os',
                        'броузер' : 'browser',
                        'время начала' : 'date_start',
                        'время окончания' : 'date_end',
                        'timestamp начала' : 'timestamp_start',
                        'timestamp окончания' : 'timestamp_end',
                        'общее время просомтра, мин' : 'total_time_views',
                        'время просомтра, мин' : 'time_views',
                        'кол-во кликов' : 'count_click',
                        'id открытой сессии' : 'id_openses',
                        'id закрытой сессии' : 'id_closeses',
                        'server session id' : 'session_id',
                        'статус открытия' : 'open_status',
                        'статус закрытия' : 'close_status',
                        'мероприятие' : 'event'})
df.columns

In [ ]:
# у нас 20130 дубликатов, удалим их
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop_duplicates(keep='first')


In [ ]:
# у нас есть пропуски в столбце user_id
df[df['user_id'].isna()]

У нас есть пустые строки, надо их удалить. 
Что касается пользователей с пропусками в user_id, то такие слушатели регистрировались по промокоду (на это указывает характерные баркоды). Баркод присваивается слушателю на конкретное мероприятие, а user_id присваивается 1 раз и он одинаков у слушателя на всех мероприятиях.

In [ ]:
# удалим те строки, где есть пропуски и в user_id и 
df = df[~((df['user_id'].isna()) & (df['barcode'].isna()))]
start_size = df.shape[0]
# избавимся от точки с 0 в данных с индификаторами пользователей
df['user_id'] = df['user_id'].fillna('-1').astype('float').astype('int').astype('str')
# заменим -1  на значение баркода
df.loc[((df['user_id'] == '-1') & (df['barcode'].notna())), 'user_id'] = df['barcode']
print('Кол-во строк в таблице df: ', start_size)

**Исследуем столбцы с временем**

In [ ]:
# посмотрим за какие дни есть данные для каждого мероприятия
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])
df.groupby('event', as_index=False).agg({'date_start' : 'min', 'date_end': 'max'})

Даты проведения **`«Мероприятие 12»`** - **с 27 апреля по 2 мая 2023 года**, **`«Мероприятие 11»`** - **1 июля по 7 июля 2022 года**. Для «Мероприятие 12» максимальная дата завершения сеанса меньше минимальной даты начала.

Преобразуем столбцы  timestamp_start и timestamp_end

In [ ]:
# из timestamp_start и timestamp_end сделаем новые столбцы, добавим 3 часа, учитывая московское время(UTC+3)
df['dt_start'] = pd.to_datetime(df['timestamp_start'], unit='s', errors='raise') + timedelta(hours=3)
df['dt_end'] = pd.to_datetime(df['timestamp_end'], unit='s', errors='raise') + timedelta(hours=3)
# создадим новый столбец с датой 'dt' в df
df['dt'] = pd.to_datetime(df['dt_start']).dt.date.astype('str')
# если строки, где время окончания меньше времени старта
df[df['dt_end'] < df['dt_start']]


In [ ]:
# в таких строках заменим  время старта на время окончания и наоборот
time = df[['dt_start', 'dt_end']]
df['dt_start'], df['dt_end'] = time.min(axis=1), time.max(axis=1)

# посмотрим на даты проведения мероприятий
df.groupby('event', as_index=False).agg({'dt_start' : 'min', 'dt_end': 'max'})

С новыми столбцами dt_start и dt_end все даты совпадают с датами проведения мероприятий.

В столбце `time_views` данные в разном формате, есть пропуски. Создадим новые столбцы и вычислим время просмотра.

In [ ]:
# создадим новые столбцы с временем просмотра для каждого входа и 
# суммарным временем за день для каждого пользователя
# пересчитаем для этих строк время просмотра
df['time_views_sec'] = (df['dt_end'] - df['dt_start']).dt.seconds
df['day_time_views_sec'] = df.groupby(['user_id', 'dt'])['time_views_sec'].transform(lambda x: x.sum())
# удалим столбцы, которые мы пересчитали
df.drop(['date_start', 'date_end', 'time_views', 'total_time_views'], axis=1, inplace=True)

In [ ]:
# распределение длительности подключений в сек
df['time_views_sec'].describe(percentiles=[.25, .5, .75, .9, .95, .99])

У нас есть подключения, которые длились 0 сек, 25 % от всех подключений длились 8 сек и меньше.  
Медиана равняется около 3 мин,  а максимальное подключение длилось 19 часов.

In [ ]:
# доля строк и пользователей с подключением  0 сек
print('Доля просмотров с длительность 0 сек: ',
      round(df[df['time_views_sec'] == 0].shape[0] / start_size * 100, 2))
print('Доля пользователей, у которых были просмотры с длительность 0 сек: ',
      round(df[df['time_views_sec'] == 0]['user_id'].nunique() / df['user_id'].nunique() * 100, 2))

In [ ]:
# посмотрим на пользователей, у которых были только просмотры с длительностью 0 сек
time_0 = (list(df[(df['user_id'].isin(df[df['time_views_sec'] == 0]['user_id'].unique()))]
                                .groupby('user_id', as_index=False)['time_views_sec'].nunique()
                                .query('time_views_sec  == 1')['user_id'].unique()))
print('Кол-во пользователей только с 0 сек просмотров: ', len(time_0))


После замены пропусков в других столбцах, удалим строки с длительностью 0 сек и дублирующие строки

In [ ]:
# посмотрим на данные с продолжительной длительностью
df[df['time_views_sec'] > np.percentile(df['time_views_sec'], 99)].sort_values(by=['user_id', 'dt_start'])

У нас есть пересекающиеся по времени просмотры у одного пользователя. У некоторых просмотры завершаются в одно время, далее посмотрим на такие строки и попытаемся их обработать.
Есть 2 просмотра, которые  начались в один день, а закончились на следующий, видимо после окончания видеотрансляции продолжалось фиксироваться время. По расписанию время окончания данной конференции в 16.00

In [ ]:
# заменим в данных строках время окончания
df.loc[(df['time_views_sec'] > 36000)
       & (df['event'] == 'Мероприятие 5'), 'dt_end'] = '2022-07-22 16:00:00'
# пересчитаем для этих строк время просмотра
df['time_views_sec'] = (df['dt_end'] - df['dt_start']).dt.seconds
df['day_time_views_sec'] = df.groupby(['user_id', 'dt'])['time_views_sec'].transform(lambda x: x.sum())


In [ ]:
# распределение суммарной длительности подключений в день у пользователя в сек
df['day_time_views_sec'].describe(percentiles=[.25, .5, .75, .9, .95, .99])

Есть пользователь, которые за день смог посмотреть 56 часов

In [ ]:
# посмотрим, на пользователей, у которых больше 16 часов просмотра в день
df[(df['day_time_views_sec'] > np.percentile(df['day_time_views_sec'], 99))\
   & (df['time_views_sec'] != 0)].sort_values(by=['user_id', 'dt_start']).head()

Пересекающие просмотры являются причиной аномально длительных просмотров за день.

In [ ]:
# посмотрим на уникальные значения потоков, кол-во кликов
for col in ['potok',  'count_click']:
    display(df.groupby('event')[col].unique().to_frame())


In [ ]:
# изменим тип данных в столбце potok на целое число
df['potok'] = df['potok'].astype('int')

В столбце `count_click` есть пропуски, возможно, что пропуски обозначают отсутствие кликов на всплывающее окно. Учитывая, что максимальное кол-во кликов в день равно 6, а у нас есть значения больше, возможно это ошибки. В этом столбце изменять ничего не будем, скорее всего он нам для исследования не пригодится.

**Исследуем столбцы с проживанием пользователей**

In [ ]:
# посмотрим на уникальные значения в столбце city, country, region
for col in ['country','region', 'city']:
    print('Уникальные значения в столбце: ' + col)
    df[col] = df[col].str.strip()
    display(df[col].sort_values().unique())

In [ ]:
# много ли строк, где в столбце с городом указаны ошибочные данные
df[df['city'].isin(['', '1', 'undefined', 'вар', 'еее', 'Выберите'])].shape[0]

In [ ]:
# посмотрим есть ли слушатели, у которых стоит указано несколько городов
several_cities = (df
 .groupby('user_id',as_index=False)['city']
 .nunique()
 .query('city > 1')['user_id'].unique()
)
print('Кол-во пользователей с несколькими городами: ' , len(several_cities))
#df[df['user_id'].isin(several_cities)]

У двоих есть в данных ошибки, например указано Ярославль с регионом Хоринск, у этого же пользователя указаны и разные профессии со специализацией. Другой указывал 1 и Ставрополь, такие ошибки исправим, у остальных 2-их оставим только  последний город

In [ ]:
# сделаем замену для таких зрителей
user_several_cities = {'19387': 'Ставрополь',
                       '8861' : 'Хоринск',
                       '1438' : 'Санкт-Петербург',
                       '6431' : 'Сестрорецк'}
df.loc[(df['user_id'].isin(several_cities)), 'city'] =\
      df.loc[(df['user_id'].isin(several_cities)), 'user_id'].map(user_several_cities)

In [ ]:
# поработаем с неявными дубликатаим в названиях стран
df['country'] = df['country'].replace('Россич', 'Россия')
# поработаем с неявными дубликатаим в названиях регионов
regions = {'Москва' : 'Москва и Московская обл.',
           'Волгоград' : 'Волгоградская обл.',
           'Санкт-Петербург': 'Санкт-Петербург и область',
           'Ленинградская область' : 'Санкт-Петербург и область',
           'Нижегородская (Горьковская)' : 'Нижегородская обл.',
           'Сахалин' : 'Сахалинская обл.',
           'Липецкая область' : 'Липецкая обл.',
           'Ульяновская':  'Ульяновская обл.',
           'Башкортостан': 'Республика Башкортостан',
           'Башкортостан(Башкирия)' : 'Республика Башкортостан',
           'Калининград' : 'Калининградская обл.',
           'Омская область' : 'Омская обл.',
           'Хоринск' : 'Бурятия',
           'Рязанская область' : 'Рязанская обл.'
           }

df['region'] = df['region'].replace(regions)

# поработаем с неявными дубликатаим в названиях городов
cities = {'Санкт – Петербург' : 'Санкт-Петербург',
          'барнаул' : 'Барнаул',
          'Ю.Сахалинск' : 'Южно-Сахалинск',
          'НижНов' : 'Нижний Новгород',
          'санкт-петербург' : 'Санкт-Петербург',
          'Бутово' : 'Москва'
         }
df['city'] = df['city'].replace(cities)

# заменим все аномальные значения на пропуски
df['country'] = df['country'].replace(['1', 'undefined', 'Выберите'], np.nan)
df['region'] = df['region'].replace(['', '1', 'undefined', 'Выберите'], np.nan)
df['city'] = df['city'].replace(['', '1', 'undefined', 'Выберите', 'вар', 'еее'], np.nan)

# сгруппируем по пользователям и заменим пропуски  на известные значения
df['city'] = df.sort_values(by='city').groupby('user_id')['city'].ffill()
df['country'] = df.sort_values(by='country').groupby('user_id')['country'].ffill()
df['region'] = df.sort_values(by='region').groupby('user_id')['region'].ffill()

# сгруппируем по городам и заменим пропуски в стране на известные значения
df['region'] = df.sort_values(by='region').groupby('city')['region'].ffill()
df['country'] = df.sort_values(by='country').groupby('city')['country'].ffill()

In [ ]:
# остались ли пропуски в стране, если есть данные о городе
df[(df['country'].isna()) & (df['city'].notna())]['city'].unique()

In [ ]:
# заменим оставшийся пропуск для города п.г.т. Кузьмоловский
df.loc[df['city'] == 'п.г.т. Кузьмоловский', 'country'] = 'Россия'

In [ ]:
# заменим пропуски для городов, у которых не указан регион
regions_city = {'Дзержинский' : 'Москва и Московская обл.',
                'Нижний Тагил': 'Свердловская обл.',
                'Химки' : 'Москва и Московская обл.',
                'п.г.т. Кузьмоловский' : 'Санкт-Петербург и область',
                'Биробиджан' : 'Еврейская автономная обл.',
                'Костанай' : 'Костанайская обл.'
               }
df.loc[(df['city'].notna()) & (df['region'].isna()), 'region'] = \
    df.loc[(df['city'].notna()) & (df['region'].isna()), 'city'].map(regions_city)

In [ ]:
# сколько осталось пропусков
df[['country', 'region', 'city']].isna().sum()

Теперь у нас одинаковое кол-во строк с пропусками по месту проживания

In [ ]:
# посмотрим сколько пользователей не указали  данные о месте проживания
print('Доля пользователей с пропусками о месте проживания (%): ',
    round(df[df['city'].isna()]['user_id'].nunique() / df['user_id'].nunique() * 100, 1))

# заменим пропуски на значение неизвестно
df[['country', 'region', 'city']] = df[['country', 'region', 'city']].replace(np.nan, 'неизвестно')

**Исследуем столбцы с профессиями и специализацией**

In [ ]:
# приведем к нижнему регистру значения в столбце 'profession' и 'specialization'
for col in ['profession','specialization']:
    df[col] = df[col].str.lower()
# посмотрим на кол-во пользователей по профессиям
df.groupby('profession', as_index=False)['user_id'].nunique().sort_values(by='user_id', ascending=False)

In [ ]:
# поправим некоторые значения в столбце с профессией
prof = {'средний медпероснал' : 'средний медперсонал',
     'мед сестра' : 'средний медперсонал',
     'фельдшер' : 'средний медперсонал',
     'немедициснкий персонал клиники' : 'немедицинский персонал клиники',
     'онколог' : 'врач-онколог',
     'врач -онколог' : 'врач-онколог',
     'онкогинеколог' : 'врач-окногинеколог',
     'онколог-гинеколог' : 'врач-окногинеколог',
     'руководитель группы проектов &quot;все не напрасно&quot;' : 'исследователь',
     'онколог , хт' : 'врач-химиотерапевт',
     'химиотерапевт' : 'врач-химиотерапевт',
     'врач, химиотерапевт': 'врач-химиотерапевт',
     'врач хт' : 'врач-химиотерапевт',
     'врач - онколог, химиотерапевт': 'врач-химиотерапевт',
     'узи' : 'врач узд',
     'руководитель хт службы 1го стационара, врач- онколог, химиотерапевт' : 'руководящее звено клиники',
     'руководитель отделения торакальной онкологии' : 'руководящее звено клиники',
     'зам гл врача' : 'руководящее звено клиники',
     'нач. мед' : 'руководящее звено клиники',
     'зам.гл.врач' :  'руководящее звено клиники',
     'зав.отд' : 'руководящее звено клиники',
     'студент' : 'обучающийся по медицинской специальности',
     'студент медицинского вуза' : 'обучающийся по медицинской специальности',
     'представитель пациентской организации' : 'представитель общественной организации',
     'нач мед' : 'руководящее звено клиники',
     'гл. онколог' : 'руководящее звено клиники',
     'х/т зав отд' : 'руководящее звено клиники',
     'руководитель хт службы 1го стационара, врач- онколог, химиотерапевт' : 'руководящее звено клиники',
     'зав. хт, химиотерапевт'  : 'руководящее звено клиники'
        }
df['profession'] = df['profession'].replace(prof)

In [ ]:
# объединим профессию и специальность в один столбец
# и посмотрим, возможно так больше будет информации о профессии и специальности
df[['profession', 'specialization']] = df[['profession', 'specialization']].fillna('')
df['prof_spec'] = (df['profession'] + ' + ' + df['specialization']).str.strip()
df.groupby('prof_spec', as_index=False)['user_id'].nunique().sort_values(by='user_id', ascending=False)

Так как в столбцах с профессиями и специализациями нет четкой структуры, каждый заполняет как хочет, то много ошибок и неявных дубликатов. Попытаемся из данных 2-х столбцов сделать столбец с профессиями зрителей. Отдельно править специализации зрителей не будем

In [ ]:
# зададим функцию
def prof_replacement(row):
    if 'ординатор - аспирант' in row:
        return 'ординатор - аспирант'
    if 'немедицинский персонал клиники' in row:
        return 'немедицинский персонал клиники'
    if 'обучающийся по медицинской специальности' in row:
        return 'обучающийся по медицинской специальности'
    if 'представитель медицинской компании' in row:
        return 'представитель медицинской компании'
    if 'исследователь' in row:
        return 'исследователь'
    if 'руководящее звено клиники' in row or 'зам.' in row or 'гл.' in row  or 'зав' in row or 'старш' in row  :
        return 'руководящее звено клиники'
    if 'врач' in row and 'хирург' in row and 'онко' in row:
        return 'врач хирург-онколог'
    if 'врач' in row and 'химиотер' in row or 'химиотерапевт' in row:
        return 'врач-химиотерапевт'
    if 'медперсонал' in row or 'перевязочная м/с' in row or 'сестра' in row:
        return 'средний медперсонал'
    if 'врач' in row and 'узд' in row:
        return 'врач ультразвуковой диагностики'
    if 'радиотерапевт' in row or ('радиотерапия' in row  and 'врач' in row):
        return 'врач-радиотерапевт'
    if ('врач' in row and 'мрт' in row) or ('врач' in row and 'рентген' in row) or 'врач + лучевая диагностика' in row:
        return 'врач-рентгенолог'
    if 'врач' in row and 'пато' in row:
        return 'врач-патологоанатом'
    if 'врач' in row and 'радиология' in row:
        return 'врач-радиолог'
    if ('врач' in row  and 'гинеколог' in row) or 'онкогинекология' in row:
        return 'врач-гинеколог'
    if 'врач' in row  and 'лаборатор' in row:
        return 'врач клинической лабораторной диагностики'
    if ('врач' in row  and 'гематоло' in row) or 'гематолог' in row:
        return 'врач-гематолог'
    if 'врач' in row  and 'эндоскоп' in row:
        return 'врач-эндоскопист'
    if 'врач' in row  and 'уролог' in row:
        return 'врач-уролог'
    if 'врач' in row  and 'колопрокт' in row:
        return 'врач-колопроктолог'
    if 'врач' in row  and ('реаниматол' in row or 'анестезиол' in row):
        return 'врач анестизиолог-реаниматолог'
    if ('врач' in row  and 'дермато' in row)  or 'дерматовенеролог' in row:
        return 'врач-дерматовенеролог'
    if 'врач' in row and 'гастро' in row:
        return 'врач-гастроэнтеролог'
    if ('врач' in row and 'нейро' in row) or 'нейрохирург' in row:
        return 'врач-нейрохирург'
    if 'врач' in row and 'пульм' in row:
        return 'врач-пульмонолог'
    if 'врач' in row and 'эндокри' in row:
        return 'врач-эндокринолог'
    if ('врач' in row and 'хирург' in row) or 'хирург' in row:
        return 'врач-хирург'
    if ('врач' in row and 'онко' in row) or 'онколог' in row:
        return 'врач-онколог'
    if 'врач' in row:
        return 'врач'
    if 'фармацевт' in row:
        return 'фармацевт'
    else:
        return 'другое'

# применим функцию, создадим новый столбец
df['profession_new'] = df['prof_spec'].apply(prof_replacement)
df.groupby('profession_new', as_index=False)['user_id'].nunique().sort_values(by='user_id', ascending=False)

В итоге удалось частично из 2-х изначальных столбцов восстановить профессии зрителей, наибольшее кол-во зрителей являются хирургами-онкологами, у 360 зрителей указано только врач и почти у 250 зрителей нет данных

**Проверим данные на неявные дубликаты и пересечения времени подключений у зрителей**

In [ ]:
# удалим данные, где время одного просмотра равна 0 сек
df = df[df['time_views_sec'] != 0]
print('Кол-во строк в таблице df: ', df.shape[0])
print('Удалили, в % ', round((1 - df.shape[0] / start_size) * 100, 2))

In [ ]:
# кол-во дубликатов
print('Кол-во дубликатов у пользователей с одинаковой датой и временем  просмотра, потоком: ',
      df[df.duplicated(subset = ['user_id', 'potok', 'dt_start', 'dt_end'])].shape[0])
# удалим дубликаты
df = df.drop_duplicates(subset = ['user_id', 'potok', 'dt_start', 'dt_end'], keep ='first')
# посмотрим если дубликаты по пользователю и времени просмотра
print('Кол-во дубликатов у пользователей с одинаковой датой и временем  просмотра: ',
      df[df.duplicated(subset = ['user_id', 'dt_start', 'dt_end'])].shape[0])

In [ ]:
# посмотрим на пересекающие по времени просмотры у пользователей
# создадим дополнительные столбцы
df['dt_next'] = df.sort_values(by=['user_id', 'dt_start']).groupby(['user_id', 'dt'])['dt_start'].shift(-1)
df['dt_previous'] = df.sort_values(by=['user_id', 'dt_start']).groupby(['user_id', 'dt'])['dt_end'].shift(1)

# выведем долю таких событий
print('Доля строк с пересекающимся временем у пользователей: ',
    round(df[(df['dt_next'] < df['dt_end']) | (df['dt_previous'] > df['dt_start'])].shape[0] / start_size * 100, 1))
print('Доля пользователей, у которых есть такие события: ',
    round(df[(df['dt_next'] < df['dt_end']) |
         (df['dt_previous'] > df['dt_start'])]['user_id'].nunique() / df['user_id'].nunique() * 100, 1))
df[(df['dt_next'] < df['dt_end']) | (df['dt_previous'] > df['dt_start'])].head()

Проанализировав нескольких пользователей и сопоставив время просмотров и расписание, а также какие сессии и доклады им было бы интресно слушать, учитывая специализацию,  можно предположить, что такие пользователи скорее всего переходили из потока в поток, но возможно не закрывали предыдущие вкладки и время окончания для всех просмотров у них зафиксировалась одно или фиксировалось с опазданием. Также если смотрели с телефона, то могли прерывать просмотр на звонок или  смотрели с нескольких устройств.

Заменим у таких пересекающихся подключений  окончание просмотра на  время начала следующего входа.

In [ ]:
# заменим время окончания на время начала следующего входа для пересекающихся строк
df = df.sort_values(by = ['user_id', 'dt_start'])
df.loc[((df['dt_next'] < df['dt_end']) | (df['dt_previous'] > df['dt_start'])), 'dt_end']\
     = df[((df['dt_next'] < df['dt_end']) | (df['dt_previous'] > df['dt_start']))].groupby(['user_id', 'dt'])['dt_start'].shift(-1)

# образовавшиеся пропуски заменим датой окончания из столбца timestamp_end
df['dt_end'] = df['dt_end'].fillna(pd.to_datetime(df['timestamp_end'], unit='s', errors='raise') + timedelta(hours=3))


In [ ]:
# посмотрим сколько пересекающихся строк осталось
df['dt_next'] = df.sort_values(by=['user_id', 'dt_start']).groupby(['user_id', 'dt'])['dt_start'].shift(-1)
df['dt_previous'] = df.sort_values(by=['user_id', 'dt_start']).groupby(['user_id', 'dt'])['dt_end'].shift(1)
# выведем долю таких событий
print('Доля строк с пересекающимся временем у пользователей: ',
    round(df[(df['dt_next'] < df['dt_end']) | (df['dt_previous'] > df['dt_start'])].shape[0] / start_size * 100, 1))
print('Доля пользователей, у которых есть такие события: ',
    round(df[(df['dt_next'] < df['dt_end']) |
         (df['dt_previous'] > df['dt_start'])]['user_id'].nunique() / df['user_id'].nunique() * 100, 1))

In [ ]:
# удалим оставшиеся пересекающиеся строки
df = df[~((df['dt_next'] < df['dt_end']) | (df['dt_previous'] > df['dt_start']))]

# пересчитаем столбцы time_views_sec и day_time_views_sec
df['time_views_sec'] = (df['dt_end'] - df['dt_start']).dt.seconds
df['day_time_views_sec'] = df.groupby(['user_id', 'dt'])['time_views_sec'].transform(lambda x: x.sum())


In [ ]:
# оставим только нужные столбцы  из таблицы df
df = df[['user_id', 'country', 'region', 'city', 'profession_new', 'dt', 'potok', 'dt_start', 'dt_end', \
         'time_views_sec', 'day_time_views_sec', 'event']]


In [ ]:
# всего удалили
print('Удалено строк, в %: ', round((1 - df.shape[0] / start_size) * 100, 1))
# кол-во пользователей
print('Уникальное кол-во пользователей: ', df['user_id'].nunique())

**Вывод:** в результате обработки удалили 8.1 % строк. В датафрейме у нас данные о 3190 зрителях.

❗️В ходе предобработки выявили следующие ошибки в данных о сборах действий зрителей:

- пропуски в данных с id зрителей
- есть строки, где время завершения просмотра меньше времени начала
- большое кол-во пропусков в личных данных зрителей, пропуски можно объяснить тем, что сами зрители не оставили о себе информацию. В данных с профессиями и специализациями много ошибок, каждый указывает, то что хочет, желательно далее при регистрации сделать данные о профессии и специализации обязательным полем и выбирать данные из предложенного словаря
- большое количество пропусков в столбцах статусом открытия и закрытия, а также в столбцах id_openses, id_closeses, количеством кликов. В этих же столбцах встречаются и аномальные значения.
- в данных у более 45% пользователей встречаются пересекающие по времени сессии, как и при просмотре одного потока так и разных. В таких сессиях не возможно понять, что в итоге смотрел зритель.
- почти 7% строк имеют длительность просмотра  0 сек. Среди 56% зрителей, у которых были строки с 0 сек, 11 зрителей имеют только такие просмотры.

### Данные с расписанием мероприятий
<a id="2"></a>

Так как не всю информацию можно опубликовывать в первоначальном виде, предобработка данных  здесь не представлена, загружено уже обработанное расписание.

In [ ]:
# посмотрим на данные с расписанием после предобработки 
df_info(df_shedule)

In [ ]:
# удалим ненужный столбец
df_shedule.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
# заменим пропуски в датафрейме
df_shedule = df_shedule.fillna(' ')
# заменим тип данных в столбцах с числом и временем
for col in ['time_start', 'time_end', 'section_start', 'section_end']:
    df_shedule[col] = pd.to_datetime(df_shedule[col], format='%Y-%m-%d %H:%M:%S')


### Объединение таблицы с данными по просмотрам и расписанием
<a id="3"></a>

In [ ]:
# объединяем таблицы
df1 = df.merge(df_shedule, on =['dt', 'potok', 'event'], how='outer')
# фильтруем
df1 = df1[(df1['dt_start'] <= df1['time_end']) & (df1['dt_end'] >= df1['time_start'])]

# кол-во пользователей 
print('Кол-во пользователей до присоединения таблиц: ', df['user_id'].nunique())
print('Кол-во пользователей после присоединения таблиц: ', df1['user_id'].nunique())
print('Доля пользователей, которые отсеились: ', round(100 - df1['user_id'].nunique() / df['user_id'].nunique()* 100, 1))

In [ ]:
df1.head()

Видимо такие пользователи подключались не вовремя чтения докладов, а раньше или позже.

`Добавим новые столбцы:`

'total_time_views_report' - длительность просмотра доклада пользователем      
'total_time_views_section' - длительность просмотра секции пользователем   
'ratio_duration_speech' - доля длительности просмотра пользователем доклада от длительности доклада    
'ratio_duration_section' - доля длительности просмотра пользователем секции от длительности секции   
'report_head' - пользователь посмотрел 50 и более % от длительности доклада

In [ ]:
# поменяем формат в столбце с датой
df1['dt'] = pd.to_datetime(df1['dt'], format='%Y-%m-%d')

# подсчет длительности просмотренного доклада для каждого просмотра пользователя
conditions = [
    (df1['dt_start'] >= df1['time_start']) & (df1['dt_end'] >= df1['time_end']),
    (df1['dt_start'] <= df1['time_start']) & (df1['dt_end'] <= df1['time_end']),
    (df1['dt_start'] <= df1['time_start']) & (df1['dt_end'] >= df1['time_end']),
    (df1['dt_start'] >= df1['time_start']) & (df1['dt_end'] <= df1['time_end'])
]
values = [(df1['time_end'] - df1['dt_start']).dt.seconds, 
          (df1['dt_end'] - df1['time_start']).dt.seconds, 
          (df1['time_end'] - df1['time_start']).dt.seconds, 
          (df1['dt_end'] - df1['dt_start']).dt.seconds]
df1['time_views_report'] = np.select(conditions, values)

In [ ]:
# сгруппируем данные
df1 = (df1
       .groupby(['dt', 'user_id', 'potok', 'title', 'time_start', 'time_end'], as_index=False)
       .agg({'speaker':'first',
             'place_of_work':'first',
             'city_speaker':'first',
             'day_time_views_sec' : 'first',
             'time_views_report' : 'sum',
             'duration_speech_sec' : 'first',
             'duration_section_sec' : 'first',
             'event': 'first',
             'country':'first',
             'region':'first',
             'city':'first',
             'profession_new':'first',
             'section_start':'first',
             'section_end':'first', 'section':'first', 'nosology':'first',
             'basic':'first'})
       .rename(columns={'time_views_report' : 'total_time_views_report'})
           )

In [ ]:
# находим длительность просмотра  сескции по каждому пользователю
df1['total_time_views_section'] = (
                                df1
                                    .groupby(['user_id', 'dt', 'potok', 'section',  'section_start', 'section_end'])
                                     ['total_time_views_report']
                                    .transform(lambda x: round(x.sum()))
                                   )
# находим долю длительности просмотра от длительности доклада 
df1['ratio_duration_speech'] = (df1['total_time_views_report'] / df1['duration_speech_sec'] * 100).round(1)

# находим долю длительности просмотра от длительности сессии 
df1['ratio_duration_section'] = (df1['total_time_views_section'] / df1['duration_section_sec'] * 100).round(1)

# если пользователь посмотрел 50 и более % от длительности доклада, то отмечаем (1), что он просмотрел доклад
df1['report_head'] = (df1['ratio_duration_speech'] >= 50.0) * 1


In [ ]:
# посмотрим на распределение доли просмотров от длительности докладов у пользователей
plt.figure(figsize=(10, 5))
plt.hist(df1['ratio_duration_speech'], color= '#486273', bins=20)
plt.xlabel('% просмотра', fontsize='12')
plt.ylabel('количество', fontsize='12')
plt.xticks(fontsize='12')
plt.yticks(fontsize='12')
plt.title('Распределение доли просмотра зрителями доклада от длительности доклада\n ', fontsize='14')
plt.show()

# применим метод describe()
df1['ratio_duration_speech'].describe().to_frame()

Видно, что чаще доклады зрители просматривают до конца. Есть у нас доклады, где доля просмотра равна нулю.

In [ ]:
# посмотрим на распределение доли просмотров секций 
plt.figure(figsize=(10, 5))
plt.hist(
    df1.groupby(['user_id', 'dt', 'section', 'potok' , 'section_start', 'section_end'])['ratio_duration_section']\
    .first(), color='#486273', bins=20)
plt.xlabel('% просмотра', fontsize='12')
plt.ylabel('количество', fontsize='12')
plt.xticks(fontsize='12')
plt.yticks(fontsize='12')
plt.title('Распределение доли просмотра зрителями секций от длительности секции\n ', fontsize='14')
plt.show()

# применим метод describe()
df1.groupby(['user_id', 'dt', 'section', 'potok' , 'section_start', 'section_end'])['ratio_duration_section']\
       .first().describe().to_frame()

Что касается просмотра секций, то зрители редко просматривают их целиком, почти половина секций просматривалась зрителем меньше 20 процентов от времени секции 

**Вывод:** после объединения таблиц, у нас остались данные просмотров  3144 зрителей, у части зрителей просмотры были вне расписания выступлений.

## Исследовательский анализ данных
<a id="eda"></a>

Посчитаем кол-во зрителей, докладов и спикеров. Нужно учесть, что для некоторых докладов стоит несколько спикеров, и есть доклады, которые, из-за отсутствия времени в расписании, мы объединяли вместе. 

In [ ]:
# посчитаем сколько было уникальных зрителей, выступлений и спикеров на каждом мероприятии
# учтем тот факт, что у  нас есть для одного доклада несколько докладчиков и совмещенные доклады
event_count = (
                df1
                    .groupby(['event', 'time_start', 'section'], as_index=False)
                    .agg({'title' : lambda x: (x.str.split(', ').explode().nunique()),
                          'speaker' : 'first'})
                    .groupby('event') 
                    .agg({'title' : 'sum',
                          'speaker' : (lambda x: x.str.split(', ').explode().nunique())})
                    .reset_index()
                    .assign(user_id =  df1.groupby('event', as_index=False)
                                          .agg({'user_id': 'nunique'})['user_id'])
                    .sort_values(by= 'user_id')
)
event_count

In [ ]:
# создадим функцию для построения графика 
def horizont_plot(data, col1, col2, s, s1):
   
    fig = px.bar(data, x=col1, y=col2, 
             text = col1, color_discrete_sequence = ['#506d80'])
    fig.update_traces(width=0.85, textposition='outside', textfont_size=12, 
                      textfont_family='Lato', textfont_color='#1d3a4d', textangle=0, cliponaxis=False)
    fig.update_traces(hovertemplate="<b>%{y}</b><br><br>"+'Кол-во: %{x}<br><extra></extra>')
    fig.update_layout(autosize=False, 
                  margin={"r":35,"t":0,"l":20,"b":20},
                  width=s,
                  height=s1,
                  xaxis=dict(title_text="", showgrid=False),
                  yaxis=dict(title_text="", showgrid=False),
                  paper_bgcolor="#f8f8f8", 
                  plot_bgcolor='#f8f8f8', 
              )

    fig.update_yaxes(tickfont=dict(family='Lato', size=12, color='#1d3a4d'), automargin=True)
    fig.update_xaxes(visible=False, fixedrange=True)
       
    return fig

In [ ]:
# тоже самое, только без вывода значений по оси y
def horizont_plot1(data, col1, col2, s,s1):
   
    fig = px.bar(data, x=col1, y=col2, 
             text = col1, color_discrete_sequence = ['#506d80'])
    fig.update_traces(width=0.85, textposition='outside', textfont_size=12, 
                      textfont_family='Lato', textfont_color='#1d3a4d', textangle=0, cliponaxis=False)
    fig.update_traces(hovertemplate='Кол-во: %{x}<br><extra></extra>')
    fig.update_layout(autosize=False, 
                  margin={"r":55,"t":0,"l":0,"b":20},
                  width=s,
                  height=s1, 
                  xaxis=dict(title_text="", showgrid=False),
                  yaxis=dict(title_text="", showgrid=False),
                  paper_bgcolor="#f8f8f8", 
                  plot_bgcolor='#f8f8f8', 
              )

    fig.update_yaxes(visible=False, fixedrange=True)
    fig.update_xaxes(visible=False, fixedrange=True)
        
    return fig

In [ ]:
# построим графики
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.layout = dbc.Row([
    html.H1("Анализ мероприятий",  style = {'font-size' : 20, 
                                            'font-weight' : 'bold', 
                                            "margin-top": "15px",
                                            "margin-left": "40px",
                                            'text-align':'left'}),
    html.Br(),
    dbc.Col([
        html.H6("Кол-во зрителей",  style = {'font-size' : 12, 
                                             'font-weight' : 'bold', 
                                             'margin-top': "30px",
                                             'margin-left': "117px",
                                             'text-align':'left'}), 
        dcc.Graph(figure=horizont_plot(event_count, "user_id" , "event", 450, 500))], width={'size': 6},
                                               style={"margin-right": "5px",  "margin-top": "0px", "margin-bottom": '30px' }),
    dbc.Col([
        html.H6("Кол-во выступлений",  style = {'font-size' : 12, 
                                            'font-weight' : 'bold', 
                                            "margin-top": "30px",
                                            "margin-left": "0px",
                                            'text-align':'left'}),
        dcc.Graph(figure=horizont_plot1(event_count, "title" , "event", 190,500))], width={'size': 3},
                          style={"margin-left": "10px",  "margin-top": "0px", "margin-bottom": '30px'}
                          ),
    dbc.Col([
        html.H6("Кол-во спикеров",  style = {'font-size' : 12, 
                                            'font-weight' : 'bold', 
                                             "margin-top": "30px",
                                             "margin-left": "0px",
                                            'text-align':'left'}),
        dcc.Graph(figure=horizont_plot1(event_count, "speaker" , "event", 150,500))], width={'size': 3},
                                      style={"margin-left": "-55px",
                                             "margin-right": "0px" ,
                                             "margin-top": "0px",
                                               "margin-bottom": '30px'}),
  ],style={"font-family" : 'Lato',
          "backgroundColor": "#f8f8f8",
          "margin-bottom": "0px",
          "margin-right": "30px",
          "margin-top": "0px"}
)

app.run_server(debug=False, use_reloader=False, port=2005) 

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%9C%D0%B5%D1%80%D0%BE%D0%BF%D1%80%D0%B8%D1%8F%D1%82%D0%B8%D1%8F.png?raw=true" width="800" height="500">

Из крупных мероприятий больше зрителей было на онкофоруме "Мероприятии 12" в 2023 году, а вот из небольших мероприятий по кол-ву зрителей лидирует с большим преимуществом  - "Мероприятие 5"


In [ ]:
# зададим функцию для построения боксплота
def box(data, col1, col2, titl, titl_x):
    fig = px.box(data, x= col1, y= col2, color_discrete_sequence = ['#486273'])
    fig.update_layout(autosize=False, 
                  margin={"r":25,"t":100,"l":0,"b":0},
                  height=500,
                  title=dict(text= '<b>'+titl+'</b>', font=dict(size=16, family='Lato', color='#1d3a4d')), 
                  xaxis=dict(title=dict(text= titl_x, 
                            font=dict(size=14, family='Lato', color='#1d3a4d')),showgrid=False),
                  yaxis=dict(title_text="", showgrid=False),
                  paper_bgcolor="#f8f8f8", 
                  plot_bgcolor='#f8f8f8', 
              )

    fig.update_yaxes(tickfont=dict(family='Lato', size=12, color='#1d3a4d'), automargin=True)
    fig.update_xaxes(tickfont=dict(family='Lato', size=12, color='#1d3a4d'), automargin=True)
    fig.show()

In [ ]:
# построим диаграмму размаха длительности просмотра в день
box((
    df1
     .groupby(['user_id', 'event', 'dt'], as_index=False) 
     .agg({'total_time_views_report': lambda x: x.sum() / 60})
    ),  'total_time_views_report', 'event', 
    'Диаграмма размаха длительности просмотров выступлений<br> зрителями в день, мин.<br>  ', 'минуты')


<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%91%D0%BE%D0%BA%D1%81%D0%BF%D0%BB%D0%BE%D1%82%20%D0%B4%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8%20%D0%BF%D1%80%D0%BE%D1%81%D0%BC%D0%BE%D1%82%D1%80%D0%B0%20%D0%B4%D0%BE%D0%BA%D0%BB%D0%B0%D0%B4%D0%BE%D0%B2.png?raw=true" width="650" height="500">

In [ ]:
# применим метод describe()
(df1
 .groupby(['user_id', 'event', 'dt'], as_index=False)
 .agg({'total_time_views_report': lambda x: x.sum() / 60})
 .groupby('event')['total_time_views_report']
 .describe()
 .reset_index()
 .sort_values(by='mean', ascending=False)
 )

На "Мероприятие 12" в 2023 году зрители в среднем больше проводили времени за просмотром  докладов в день.
В целом, чем меньше докладов на мероприятии, тем меньше и среднее время просмотра.


In [ ]:
# построим диаграмму размаха кол-ва просмотренных докладов зрителями в день
# доклад считается просмотренным, если зритель просмотрел половину и более доклада 
box((df1
    .groupby(['user_id', 'event', 'dt'], as_index=False)['report_head'].sum()),
    'report_head', 'event',  
    'Диаграмма размаха количества просмотренных докладов<br> зрителями в день<br>  ', 'кол-во докладов'
    )


<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%91%D0%BE%D0%BA%D1%81%D0%BF%D0%BB%D0%BE%D1%82%20%D0%BA%D0%BE%D0%BB-%D0%B2%D0%B0%20%D0%BF%D1%80%D0%BE%D1%81%D0%BC%D0%BE%D1%82%D1%80%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B4%D0%BE%D0%BA%D0%BB%D0%B0%D0%B4%D0%BE%D0%B2.png?raw=true" width="650" height="500">

In [ ]:
# применим метод describe()
(df1
 .groupby(['user_id', 'event', 'dt'], as_index=False)
 .agg({'report_head': 'sum'})
 .groupby('event')['report_head']
 .describe()
 .reset_index()
 .sort_values(by='mean', ascending=False)
)

На "Мероприятие 12" зрители в среднем смотрели почти 10 докладов в день, на "Мероприятие 11" почти на 2 доклада меньше. 

Далее посмотрим  как по дням распределялась активность зрителей для 2-х этих мероприятий

In [ ]:
# создадим отдельную таблицу для онкофорумов Белые ночи
forum = df1[df1['event'].isin(['Мероприятие 11', 'Мероприятие 12'])]
# создадим столбец с номером дня проведения форума
forum['number_day'] = forum.sort_values(by='dt').groupby('event')['dt'].rank(method='dense').astype('int')

# сгруппируем данные  и посчитаем кол-во зрителей, докладов и спикеров
forum_day = (
            forum[forum['speaker'] != '']
                .groupby(['event', 'number_day', 'time_start', 'section'], as_index=False)
                .agg({'title' : lambda x: (x.str.split(',').explode().nunique()),
                      'speaker' : 'first'})
                .groupby(['event', 'number_day']) 
                .agg({'title' : 'sum',
                      'speaker' : (lambda x: x.str.split(',').explode().nunique())})
                .reset_index()
                .assign(user_id =  forum.groupby(['event', 'number_day'], as_index=False)
                                        .agg({'user_id': 'nunique'})['user_id'])
                .sort_values(by= ['number_day', 'event'])
)
forum_day

In [ ]:
# определим цвета для мероприятий
colors ={
         'Мероприятие 12' : '#80506d',
         'Мероприятие 11' : '#91b1ae',
         'другие мероприятия': '#d7d1a0'
        }

In [ ]:
# создадим функцию для отрисовки линейного графика
def plotly_line(data,col1, col2, titl, y1, y2):
    fig = px.line(data, x= col1, y=col2,
                  color='event', markers=True, 
                  color_discrete_map = colors
                 )
    fig.update_traces(line=dict(width=3.2), marker=dict(size=8))
    fig.update_traces(hovertemplate='<br>День: %{x}<br>Количество: %{y}<br><extra></extra>')
    fig.update_layout(autosize=True, 
                  margin={"r":25,"t":120,"l":30,"b":20},
                  width=650,
                  height=400,
                  title=dict(text='<b>'+titl+'</b>',  y = 0.95,
                            font=dict(size=18, family='Lato', color='#1d3a4d')), 
                  xaxis=dict(title=dict(text= "день", 
                            font=dict(size=14, family='Lato', color='#1d3a4d')), showgrid=False),
                  yaxis=dict(title=dict(text= "количество", 
                            font=dict(size=14, family='Lato', color='#1d3a4d')), showgrid=False, 
                            ),
                  
                  paper_bgcolor="#f8f8f8", 
                  plot_bgcolor='#f8f8f8', 
                  legend=dict(title='', orientation ='h', y=1.28, xanchor="left",  
                                   x=-0.02,  
                              font_size=10, font_color='#1d3a4d')
              )

    fig.update_yaxes(range=[y1, y2], linewidth=1, linecolor='grey',
                 tickfont=dict(family='Lato', size=12, color='#1d3a4d'), automargin=True)
    fig.update_xaxes(linewidth=1, linecolor='grey',
                  tickfont=dict(family='Lato', size=14, color='#1d3a4d'), automargin=True)
    return fig

In [ ]:
# применим функцию
plotly_line(forum_day, "number_day", "title", 'Распределение докладов по дням', 0, 250)

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%9A%D0%BE%D0%BB-%D0%B2%D0%BE%20%D0%B4%D0%BE%D0%BA%D0%BB%D0%B0%D0%B4%D0%BE%D0%B2%20%D0%BF%D0%BE%20%D0%B4%D0%BD%D1%8F%D0%BC.png?raw=true" width="650" height="400">

Видно, что расписание отличается, в 2023 году в первый день было 17 докладов на одном потоке, а вот в следующие дни уже трансляция делилась на потоки и кол-во докладов резко возросло, в 2022 году кол-во докладов было распределено примерно равномерно. В 2023 году максимальное кол-во докладов пришлось на пятницу, в 2022 году на вторник.

In [ ]:
# применим функцию
plotly_line(forum_day, "number_day", "user_id", 'Распределение зрителей по дням', 100, 1150)


<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%9A%D0%BE%D0%BB-%D0%B2%D0%BE%20%D0%B7%D1%80%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B8%CC%86%20%D0%BF%D0%BE%20%D0%B4%D0%BD%D1%8F%D0%BC.png?raw=true" width="650" height="400">

Максимальное кол-во зрителей в 2022 году пришлось на 1 день и составило - 1007, в 2023 году- на 2-й день -916 зрителей. На выходных кол-во зрителей резко снижается. 

In [ ]:
# применим функцию
plotly_line((forum[(forum['speaker'] != '')]
     .groupby(['user_id', 'event', 'number_day'], as_index=False)['report_head'].sum()
     .groupby(['event', 'number_day'], as_index=False)['report_head'].mean().round(1)
), "number_day", "report_head", 'Среднее количество просмотренных докладов', 0, 20)

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%B5%20%D0%BA%D0%BE%D0%BB-%D0%B2%D0%BE%20%D0%BF%D1%80%D0%BE%D0%BC%D0%BE%D1%82%D1%80%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B4%D0%BE%D0%BA%D0%BB%D0%B0%D0%B4%D0%BE%D0%B2.png?raw=true" width="650" height="400">

In [ ]:
# посчитаем кол-во зрителей, кол-во просмотров, глубину просмотра на доклад
# исключим 'Доклад 1398' - это вступительное слово
def titles_tab(data):
    titles_count_users = (
                 data[data['title'] != 'Доклад 1398']
                    .groupby(['dt', 'event', 'title', 'time_start', 'time_end', 'speaker', 'section'], 
                             as_index=False)
                    .agg(count_users= ('user_id','nunique'),
                         number_views=('report_head','sum') ,
                         mean_depth= ('ratio_duration_speech', lambda x: round(x.mean(),1)),
                         population_prof= ('profession_new', pd.Series.mode))
                    .assign(rate = lambda x: round(x['number_views'] / x['count_users'] * 100, 1))
                    .sort_values(by= 'number_views', ascending=False)
    )

    wrapper1 = textwrap.TextWrapper(width=15, max_lines=2, placeholder='...',  subsequent_indent='<br>')
    titles_count_users['title_1'] = titles_count_users['title'].apply(lambda x: wrapper1.fill(x)) 
    return titles_count_users

In [ ]:
titles_count_users = titles_tab(df1)
titles_count_users.head()

In [ ]:
# общие средние значения по докладам
print("Среднее кол-во зрителей на доклад: ", titles_count_users['count_users'].mean().round(1))
print("Среднее кол-во просмотров на доклад: ", 
      titles_count_users['number_views'].mean().round(1))
print("Средняя глубина просмотра на доклад, %: ", titles_count_users['mean_depth'].mean().round(1))


In [ ]:
# глубина просмотра доклада
titles_count_users.groupby('event')['mean_depth'].describe().reset_index()

In [ ]:
# создадим функцию для отрисовки горизонтального барплота с заголовком
def horizont_plot2(data, col1, col2, titl, n1,n2):
   
    fig = px.bar(data, x=col1, y=col2,  orientation='h',
             text = col1, color_discrete_sequence = ['#506d80']
                )
    fig.update_traces(width=0.87, textposition='outside', textfont_size=13, 
                      textfont_family='Lato', textfont_color='#1d3a4d', textangle=0, cliponaxis=False)
    fig.update_traces(hovertemplate="<b>%{y}</b><br><br>"+'Кол-во просмотров: %{x}<br><extra></extra>')
    fig.update_layout(autosize=True, 
                  margin={"r":50,"t":100,"l":20,"b":10},
                  width=n1,
                  height=n2,
                  title=dict(text='<b>'+ titl +'</b>', x=0.08, y=0.95,
                            font=dict(size=20, family='Lato', color='#1d3a4d')), 
                  xaxis=dict(title_text="", showgrid=False),
                  yaxis=dict(title_text="", showgrid=False),
                  paper_bgcolor="#f8f8f8", 
                  plot_bgcolor='#f8f8f8', 
              )

    fig.update_yaxes(tickfont=dict(family='Lato', size=13,  color='#1d3a4d'), automargin=True)
    fig.update_xaxes(visible=False, fixedrange=True)
    return fig

In [ ]:
horizont_plot2(titles_count_users.iloc[0:10].sort_values(by='number_views'), "number_views" , "title_1",
               'Топ-10 докладов<br>по количеству просмотров<br>  <br> ' , 450, 600)


<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%A2%D0%BE%D0%BF%2010%20%D0%B4%D0%BE%D0%BA%D0%BB%D0%B0%D0%B4%D0%BE%D0%B2%20%D0%BF%D0%BE%20%D0%BA%D0%BE%D0%BB-%D0%B2%D1%83%20%D0%BF%D1%80%D0%BE%D1%81%D0%BC%D0%BE%D1%82%D1%80%D0%BE%D0%B2.png?raw=true" width="450" height="600">

### Анализ спикеров

In [ ]:
# посчитаем кол-во выступлений, кол-во всех зрителей, кол-во просмотров, ср. кол-во просмотров
# удалим 'Спикер 968' - это пропуск
def speakers_tab(data):
    speaker_count_users = (
                 data[data['speaker'] != 'Спикер 968']
                    .groupby(['speaker', 'place_of_work', 'time_start'], as_index=False)
                    .agg({'user_id' : 'nunique', 
                          'title' : 'nunique',
                          'report_head' : 'sum'})
                    
                     .groupby(['speaker', 'place_of_work'], as_index=False)
                     .agg({'title' : 'sum',
                           'user_id' : 'sum', 
                           'report_head' : 'sum'})
                     .assign(rate = lambda x: round(x['report_head'] / x['user_id'] * 100, 1),
                             mean_count = lambda x: round(x['report_head'] / x['title']).astype('int'))
                    
                    .sort_values(by= 'report_head', ascending=False)
     )
     
    return  speaker_count_users

In [ ]:
speaker_count_users = speakers_tab(df1)
speaker_count_users.head()

In [ ]:
# построим график 
horizont_plot2(speaker_count_users.iloc[0:10].sort_values(by='report_head'), "report_head" , "speaker",
               'Топ-10 спикеров<br>по количеству просмотров<br>  <br> ' , 450, 600)

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%A2%D0%BE%D0%BF-10%20%D1%81%D0%BF%D0%B8%D0%BA%D0%B5%D1%80%D0%BE%D0%B2.png?raw=true" width="450" height="600">



In [ ]:
# построим график 
horizont_plot2(
    speaker_count_users.sort_values(by='mean_count', ascending=False).iloc[0:10].sort_values(by='mean_count'),
    "mean_count" , "speaker",
    'Топ-10 спикеров<br>по среднему количеству просмотров<br>  <br> ' , 450, 600)

<img src="https://raw.githubusercontent.com/KristinaChu/picture/41a183afe411d6b57d80c7a55cc80c6e899f6262/oncoforum/%D0%A2%D0%BE%D0%BF-10%20%D1%81%D0%BF%D0%B8%D0%BA%D0%B5%D1%80%D0%BE%D0%B2%20%D0%BF%D0%BE%20%D1%81%D1%80.%20%D0%BA%D0%BE%D0%BB-%D0%B2%D1%83%20%D0%BF%D1%80%D0%BE%D1%81%D0%BC%D0%BE%D1%82%D1%80%D0%BE%D0%B2.png" width="450" height="600">

Некоторые спикеры  выступали несколько раз, поэтому лидеры по общему кол-ву зрителей и среднему кол-ву зрителей отличаются.

**Анализ секций**

In [ ]:
# кол-во зрителей по каждой секции, кол-во зрителей, которые посмотрели хотя бы 1 доклад в секции
# ср. кол-во просмотренных докладов в секции зрителями и ср. глубину просмотра

def sections_tab(data):
    section_count_users = (
                 data
                    .groupby(['user_id', 'dt', 'section', 'section_start', 'event'], as_index=False)
                    .agg({'ratio_duration_section' : 'first',
                          'report_head' : 'sum',
                          'basic': 'first'})
                     .groupby(['dt', 'section', 'section_start', 'event', 'basic'], as_index=False)
                     .agg(
                          count_users=('user_id', 'nunique'), 
                          count_number_views_titles=('report_head','sum'),
                          mean_number_views= ('report_head','mean'),
                          mean_depth= ('ratio_duration_section','mean'))
                    .round(1)
                    .assign(count_titles = (data
                                                .groupby(['dt', 'section', 'section_start'], as_index=False)
                                                .agg({'time_start':'nunique'})['time_start']))                  
                    .sort_values(by= 'count_users', ascending=False)
                   
 )
    # добавляем новые столбцы, нужные для графиков
    section_count_users['event_new'] = (np.where(section_count_users['event']
                                        .isin(['Мероприятие 11', 'Мероприятие 12']),
                                        section_count_users['event'], 'другие мероприятия'))
   
    return section_count_users

In [ ]:
# применим функцию sections_tab
section_count_users = sections_tab(df1)
section_count_users.head()

In [ ]:
# общие средние значения по секциям
print("Среднее кол-во выступлений для всех секций: ", section_count_users['count_titles'].mean().round(1))
print("Среднее кол-во просмотренных выступлений в секции: ", 
      section_count_users['mean_number_views'].mean().round(1))
print("Средняя глубина просмотра для всех секций, %: ", section_count_users['mean_depth'].mean().round(1))


In [ ]:
# построим диаграмму размаха глубины просмотра доклада
box((section_count_users[section_count_users['event'].isin([
    'Мероприятие 11', 'Мероприятие 12'])]),
    'mean_depth', 'event',  
    'Диаграмма размаха глубины просмотра секций<br>  ', "%"
    )

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%91%D0%BE%D0%BA%D1%81%D0%BF%D0%BB%D0%BE%D1%82%20%D0%B3%D0%BB%D1%83%D0%B1%D0%B8%D0%BD%D1%8B%20%D0%BF%D1%80%D0%BE%D1%81%D0%BC%D0%BE%D1%82%D1%80%D0%B0%20%D1%81%D0%B5%D0%BA%D1%86%D0%B8%D0%B8%CC%86.png?raw=true" width="650" height="500">

В 2022 году на "Мероприятии 11" секции смотрели более продолжительно, возможно из-за того, что самих секций было меньше и зрители реже переключались с одного потока на другой.

In [ ]:
# функция для скатерплота
def scatter_chart(data, col1, col2, col3, col4, w, h):
    fig = px.scatter(
              data, 
              x=col1, y=col2, 
              color=col3, size=col4, 
              color_discrete_map = colors,
              size_max=35,
              custom_data=['count_titles', 'event_new', 'section'])
    fig.update_traces(marker_sizemin = 4)
    fig.update_traces(
    hovertemplate="<b>%{customdata[2]}</b><br><br>Кол-во зрителей: %{y}<br>Глубина просмотра, %: %{x:.1f}<br>Кол-во докладов: %{customdata[0]} <br>Ср.кол-во просмотренных докладов: %{marker.size:.1f}<extra></extra>")
    fig.update_layout(autosize=True, 
                  margin={"r":15,"t":120,"l":40,"b":10},
                  width=w,
                  height=h,
                  title=dict(text='<b>'"Анализ секций по количеству зрителей, глубине просмотра и среднего количества<br>просмотренных докладов<br> "'</b>', 
                             y = 0.97, x=0.01,
                            font=dict(size=20, family='Lato', color='#1d3a4d')), 
                  xaxis=dict(title=dict(text= "глубина просмотра, %", 
                            font=dict(size=14, family='Lato', color='#1d3a4d')), showgrid=False),
                  yaxis=dict(title=dict(text= "количество зрителей", 
                            font=dict(size=14, family='Lato', color='#1d3a4d')), showgrid=False),
                  paper_bgcolor="#f8f8f8", 
                  plot_bgcolor='#f8f8f8', 
                  legend=dict(title='', orientation ='v', y=1.11, xanchor="left",  
                                  x=0.05,  
                              font_size=12, font_color='#1d3a4d')
              )

    fig.update_yaxes(tickfont=dict(family='Lato', size=12, color='#1d3a4d'), automargin=True)
    fig.update_xaxes(tickfont=dict(family='Lato', size=14, color='#1d3a4d'), automargin=True)
    return fig

In [ ]:
scatter_chart(section_count_users, "mean_depth", "count_users",  "event_new", 'mean_number_views', 900, 600)

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%A1%D0%B5%D0%BA%D1%86%D0%B8%D0%B8.png?raw=true" width="800" height="600">

**Посмотрим секции по тематикам и нозологии**

In [ ]:
# у нас были секции  с пропусками, сделаем замену на 'мультидисциплинарное'
df1['basic'] = df1['basic'].str.lower().str.strip().replace('', 'мультидисциплинарное')
# посмотрим сколько было секций в разной тематики
df1.groupby('basic')['section_start'].nunique().sort_values(ascending=False).reset_index()


In [ ]:
# посмотрим сколько было секций в разной нозологии
df1['nosology'] = df1['nosology'].str.lower().str.strip().replace('', 'без нозологии')
df1.groupby('nosology')['section_start'].nunique().sort_values(ascending=False).reset_index()


### Анализ по городам и профессиям

In [ ]:
horizont_plot2(df1.groupby('city')['user_id'].nunique().sort_values(ascending=False).reset_index().head(10)\
               .sort_values(by='user_id'),
    "user_id" , "city",
    'Топ-10 городов<br>по количеству зрителей<br>  <br> ' , 400, 500)


<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%A2%D0%BE%D0%BF-10%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2.png?raw=true" width="400" height="500">

In [ ]:
horizont_plot2(df1.groupby('profession_new')['user_id'].nunique().sort_values(ascending=False).reset_index().head(10).sort_values(by='user_id'),
    "user_id" , "profession_new",
    'Топ-10 профессий<br>по количеству зрителей<br>  <br> ' , 500, 550)

<img src="https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%A2%D0%BE%D0%BF-10%20%D0%BF%D1%80%D0%BE%D1%84%D0%B5%D1%81%D1%81%D0%B8%D0%B8%CC%86.png?raw=true" width="500" height="550">

Далее выберем топ-15 профессий, и посмотрим сколько человек данной профессии смотрели ту или иную тематику

In [ ]:
# выбираем профессии, также включим зрителей, которые не укази свою профессию
prof_top = (
    df1
        .groupby('profession_new')['user_id']
        .nunique()
        .sort_values(ascending = False)
        .reset_index().head(16)['profession_new'].tolist()
 )


# будем учитывать зрителя, если он посмотрел хотя бы один доклад по данной тематики
# добавим столбец с кол-вом уникальных зрителей в каждой профессии и посчитаем долю

df_prof_top = (df1[(df1['profession_new'].isin(prof_top)) & (df1['report_head'] == 1)]
               .groupby(['profession_new','basic'])['user_id'].nunique().reset_index()
                .merge((df1[df1['profession_new'].isin(prof_top)] 
                           .groupby('profession_new')
                           .agg(total_users_prof = ('user_id', 'nunique'))
                           .reset_index()), on='profession_new', how='left')
                .assign(rate = lambda x: (x['user_id'] / x['total_users_prof'] * 100).round(1) )
                .sort_values(by='total_users_prof', ascending=False))
display(df_prof_top.head())

# столбцы для графика
wrapper_prof = textwrap.TextWrapper(width=25)
df_prof_top['profession_new_1'] = df_prof_top['profession_new'].apply(lambda x: wrapper_prof.fill(x))     
df_prof_top['basic_1'] = df_prof_top['basic'].apply(lambda x: wrapper_prof.fill(x))

In [ ]:
# построим тепловую карту  
plt.figure(figsize=(15, 8))

sns.heatmap((df_prof_top
             .sort_values(by='total_users_prof', ascending=False)
             .pivot(index='basic_1', columns='profession_new_1', values='rate')
             .fillna(0).T),
              annot = True,  cbar=False, cmap=['#182026','#202b33','#283640','#30414c','#384c59','#405766','#486273','#506d80', '#617b8c', '#728a99', '#8498a6', '#96a7b2', '#a7b6bf',
                  '#b9c4cc', '#cad3d8', '#dce1e5', '#edf0f2'][::-1],
              fmt='.0f')
    

plt.xlabel('')
plt.xticks(fontsize='13')
plt.ylabel('')
plt.yticks(fontsize='13')
plt.title('Доля зрителей  разных специальностей в различных тематиках\n ', fontsize='16')
plt.show()

Самые популярные тематики у зрителей всех профессий это мультидисциплинарное, лекарственная терапия и хирургия. Интересно, что из 529 врачей онкологов-хирургов только 63% посмотрели хотя бы один доклад в тематике по хирургии, а доклады из секций с мультидисциплинарной, посмотрело больше 69%. Хирургическая секция также популярна у врачей-химиотерапевтов, 60% из них посмотрели хотя бы один доклад по хирургии. 

In [ ]:
# создадим функцию для отображения индикатора
def indicator_chart(value):
    fig = go.Figure(go.Indicator(
                 mode = "number",
                 number={'font_color':'#57baae', 'font_size':40},
                 value = value
                    ))
       
    fig.update_layout(paper_bgcolor = '#f8f8f8', margin={"r":0,"t":0,"l":0,"b":0}, height=50)
    return fig

## Дашборд
<a id="dash"></a>

In [ ]:
# дашборд

app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div([
     # заголовок
        dbc.Row([
            dbc.Col(
                html.Img(id="player_image1", 
                     src= 'https://upload.wikimedia.org/wikipedia/commons/b/be/Petrov_Research_Institute_of_Oncology_Logo.png?uselang=ru',
                     style={"height": "40px",
                             "width": "auto",
                             "margin-bottom": "10px",
                             "margin-top": "15px"}),
                       width={'size' : 1}, style={"backgroundColor": "#254961",  
                                                  "margin-bottom": "0px",
                                                  "margin-top": "0px"
                                                 }),
            dbc.Col([
                html.Div("НМИЦ онкологии",  style = {'font-size' : 16 , 
                                                     'font-weight' : 'bold', 
                                                     'margin-left' : "-65px",
                                                     "margin-top": "15px"}),
                html.H6("им. Н.Н.Петрова", style = {'font-size' : 14, 
                                                    'margin-left' : "-65px"}),
                html.Br()],
                   width={'size' : 1}, style={'backgroundColor': "#254961",
                                              'color': '#f8f8f8',  
                                              "margin-top": "0px",
                                              'font-family' :'Lato'}),
                
           dbc.Col([ 
                html.H1("Анализ трансляций",  style = {'font-size' : 30, 
                                                       'font-weight' : 'bold', 
                                                       "margin-top": "20px",
                                                       "margin-left": "0px",
                                                       'text-align':'center',
                                                       'color': "#1d3a4d"})],
                                                  
                     width={'size' : 10}, style={"margin-bottom": "0px",
                                                 "margin-top": "0px",
                                                 'color': "#254961",
                                                 'font-family' :'Lato',
                                                 "backgroundColor": "#eef8f6"}
            )
        ],  className="h-15", style={"margin-bottom": "0px",
                                    "margin-top": "10px",
                                    'margin-left' : "0px",
                                    'margin-right' : "0px"}
                           ),

    dbc.Row([
         # добавление фильтров
    
        dbc.Col([
                html.Br(),
                html.Div([html.Img(id="player_image2", 
                                  src="https://img.icons8.com/ios-filled/50/EBEBEB/filter--v1.png",
                    style={ "height": "18px",
                             "width": "auto",
                             "margin-left": "5px",
                             "margin-top": "0px",
                             })], style={'display': 'inline-block'}),
                html.Div([ html.H6('Выберите фильтр', style={'font-size':14,
                                                             'color': "#57baae", 
                                                             "margin-left": "13px",
                                                             'font-family' :'Lato', 
                                                             "margin-top": "10px"
                          
                          })], style={'display': 'inline-block'}),
            
                html.Details([
                    html.Div([
                        dcc.Checklist(
                        id="event-checklist",
                        options= df1['event'].sort_values().unique(),
                        value = df1['event'].sort_values().unique(),
                        labelStyle={'color': '#black',  
                                    "backgroundColor": "#f8f8f8",
                                    'font-family' :'Lato', 
                                    'font-size':12, 
                                    'margin-left': '1px'}
                                            )
               ],
                    className="updates-list", style={"overflow-x":'hidden'}
        ),  
                html.Summary(
                     html.Code(f"Мероприятие", style={"color":"#f2f2f2",
                                                      'font-family' :'Lato', 
                                                      'font-size':20,
                                                    
                                                    }),
                        className="updates-header", style={"color": "#57baae", 
                                                           'font-size':28, 
                                                           'margin-left' : "10px", 
                                                            "margin-top": "10px", 
                                                            'font-family' :'Lato'})]
        ),
                html.Details([
                    html.Div([dcc.DatePickerRange(
                       id='date-picker',
                       min_date_allowed= df1['dt'].min(),
                       max_date_allowed= df1['dt'].max(),
                       initial_visible_month=df1['dt'].min(),
                       start_date=df1['dt'].min(),
                       end_date=df1['dt'].max(),
                       display_format='D.M.Y',
                       )], style={"color": "#254961", 
                                  'font-family' :'Lato', 
                                  'font-size':10,
                                  "backgroundColor": "#254961", 
                                  'width':'100%', 
                                  'height':'80%'}),
                html.Summary(
                    html.Code(f"Дата", style={"color":"#f2f2f2",
                                              'font-family' :'Lato', 
                                              "margin-top": "10px",
                                              'font-size':20 }),
                     style={"color": "#57baae", 
                            'font-size':28, 
                            'margin-left' : "10px",  
                            "margin-top": "10px",
                            'font-family' :'Lato'}, className="updates-header" )      
                   
        ]),
          
    
                html.Br(),
                html.Br(),
                html.Div(
                    [html.Button("Выбрать", id="events-botton", n_clicks=0)],
                     className="me-1",   
                     style={"display": "flow-root", 'margin-left' : "60px", 'font-size':18,  "color":"#254961"}
    ),
                html.Br()
            
        ],  width={'size': 2} , style={"backgroundColor": "#254961", 
                                       "margin-bottom": "-10px",
                                       "margin-top": "0px",
                                       'font-family' :'Lato',}),
       
        dbc.Col([
            dbc.Tabs([
             # отрисовка 1 страницы дашборда
                dbc.Tab([
                    dbc.Row([
                        dbc.Col([
                            html.Br(),
                            # добавление индикаторов
                            html.H6( "Количество уникальных зрителей", style = {'font-size' : 18, 
                                                                                 'color': "#1d3a4d",
                                                                                 "margin-top": "10px",
                                                                                 "margin-right": "50px",
                                                                                 "margin-left": "20px",
                                                                                 'text-align':'left'}),
                   
                            dcc.Graph(id='count_users', 
                                      style = {"margin-top": "10px",
                                               'margin-left': "-23px"} )], width={'size': 3}, 
                                      style = {"backgroundColor": '#f8f8f8',
                                                "margin-top": "30px",
                                                "margin-left": "10px",
                                                'margin-bottom': "250px"}),  
                        dbc.Col([html.Br(),
                            html.H6( "Количество уникальных спикеров", style = {'font-size' : 18, 
                                                                                 'color': "#1d3a4d",
                                                                                 "margin-top": "10px",
                                                                                 "margin-left": "20px",
                                                                                 "margin-right": "50px",
                                                                                 'text-align':'left'}),
                            dcc.Graph(id='count_speakers', 
                                      style = {"margin-top": "10px",
                                               'margin-left': "-23px"})], width={'size': 3},
                                               style = {"backgroundColor": '#f8f8f8',
                                                        "margin-top": "30px",
                                                        "margin-left": "5px",
                                                        'margin-bottom': "250px"}),
                        dbc.Col([html.Br(),
                            html.H6("Количество выступлений", style = {'font-size' : 18, 
                                                                        'color': "#1d3a4d",
                                                                        "margin-top": "10px",
                                                                        "margin-left": "20px",
                                                                         "margin-right": "50px",
                                                                        'text-align':'left'}),
                            dcc.Graph(id='сount_titles', 
                                      style = {"margin-top": "10px",
                                               'margin-left': "-23px"})], width={'size': 3},
                                               style = {"backgroundColor": '#f8f8f8',
                                                        "margin-left": "5px",
                                                        "margin-top": "30px",
                                                        'margin-bottom': "250px"}),
                        dbc.Col([html.Br(),
                            html.H6( "Количество секций", style = {'font-size' : 18, 
                                                                    'color': "#1d3a4d",
                                                                    "margin-top": "10px",
                                                                    "margin-right": "130px",
                                                                    "margin-left": "20px",
                                                                    'text-align':'left'}),
                            dcc.Graph(id='сount_sections', 
                                      style = {"margin-top": "10px",
                                               'margin-left': "-23px"})],  width={'size': 3},
                                      style = {"backgroundColor": '#f8f8f8',
                                                "margin-top": "30px",
                                                "margin-left": "-30px",
                                                'margin-bottom': "250px"})
                            
                        ], className="h-15", style={'margin-bottom' : "0px", 
                                                    'margin-top' : "0px", 
                                                    'margin-left': "0px", 
                                                    "margin-right": "0px"}
                    ),
                    dbc.Row([
                        dbc.Col([
                            html.Br(),
                            html.H6( "Cредняя длительность просмотра, мин. ",
                                    style = {'font-size' : 18, 
                                             'color': "#1d3a4d",
                                             "margin-top": "25px",
                                             "margin-left": "20px",
                                             "margin-right": "50px",
                                             'text-align':'left'}),
                            dcc.Graph(id='mean_users_view', 
                                      style = {"margin-top": "10px",
                                               'margin-left': "-18px"}),
                            html.Br(),
                            html.Br(),
                            html.Br(),
                            html.Br(),
                            html.H6( "Cреднее количество просмотренных докладов",
                                    style = {'font-size' : 18, 
                                             'color': "#1d3a4d",
                                              "margin-top": "0px",
                                              "margin-left": "20px",
                                              "margin-right": "50px",
                                              'text-align':'left'}),
                            dcc.Graph(id='mean_users_titles', 
                                      style = {"margin-top": "10px",
                                               'margin-left': "-18px"})], width={'size': 3},
                                      style = {"backgroundColor": '#f8f8f8',
                                                "margin-top": "-200px",
                                                "margin-left": "15px",
                                                'margin-bottom': "20px"}),
                         # добавление календаря мероприятий
                        dbc.Col([
                            html.H6("Календарь мероприятий",
                                    style = {'font-size' : 18, 
                                             'color': "#1d3a4d",
                                              "margin-top": "15px",
                                              "margin-left": "15px",
                                              "margin-right": "50px",
                                              'text-align':'left'}),
                             dcc.Graph(id='calendar')]
                        ,  width={'size': 7}, style={'margin-bottom' : "20px", 
                                                     'margin-top' : "-200px", 
                                                     'margin-left': "10px" }), 
                       ], className="h-60", style={'margin-bottom' : "0px", 
                                                   'margin-top' : "0px" }
                    
                    ),   
                
                 
                 dbc.Row([
                    html.H6('ℹ︎ доклад считается просмотренным, если зритель посмотрел половину и более его длительности',
                                    style = {'font-size' : 12, 
                                             'color': "#909090",
                                             "margin-top": "-20px",
                                             "margin-left": "-5px",
                                             "margin-right": "40px",
                                             'margin-bottom' : "5px",
                                             'text-align':'right'})], className="h-2"),],  
                    label="Общее ⎘"),
                # отрисовка 2 страницы дашборда
                dbc.Tab([
                    dbc.Tabs([
                        dbc.Tab([
                            dbc.Row([
                                dbc.Col([
                                    dcc.Graph(id='titles_top')], width={'size': 5},
                                           style={'margin-bottom' : "10px", 
                                                  'margin-top' : "0px", 
                                                  'margin-left': "25px" }),
                                dbc.Col([
                                    dcc.Graph(id='speaker_top')],  width={'size': 5},
                                              style={'margin-bottom' : "0px",
                                                     'margin-top' : "10px", 
                                                     'margin-left': "0px" }),
                       ], className="h-60", style={'margin-bottom' : "0px", 
                                                   'margin-top' : "10px" })], label="График"),
                    
                    dbc.Tab([
                            dbc.Row([
                                html.Br(),
                                html.Div(id= 'table_titles')],
                                        style={'margin-bottom' : "0px", 
                                               'margin-top' : "0px", 
                                               'margin-left': "10px" }),
                    
                ], label="Таблица 1"),
                    dbc.Tab([
                            dbc.Row([
                                html.Br(),
                                html.Div(id= 'table_speakers')],
                                         style={'margin-bottom' : "0px", 
                                                'margin-top' : "0px", 
                                                'margin-left': "10px" })], label="Таблица 2")])
                ], label="Выступления ⎘"),
            # отрисовка 3 страницы дашборда
            dbc.Tab([
                    dbc.Tabs([
                        dbc.Tab([
                            dbc.Row([
                                dbc.Col([
                                    dcc.Graph(id='section_scatter')], width={'size': 10},
                                              style={'margin-bottom' : "10px", 
                                                     'margin-top' : "0px", 
                                                     'margin-left': "50px" }),
                                 
                                
                       ], className="h-70", style={'margin-bottom' : "0px", 
                                                   'margin-top' : "30px" })], 
                            label="График"
                    ),
                    
                    dbc.Tab([
                            dbc.Row([
                                html.Br(),
                                html.Div(id= 'table_section')],
                                         style={'margin-bottom' : "0px", 
                                                'margin-top' : "0px", 
                                                'margin-left': "10px" }),
                    
                ], label="Таблица")])
                   
                ], 
                    label="Секции ⎘")
                  ])
        ],  width={'size': 10})
            
    ], className="h-85", style={'margin-left' : "0px", 'margin-right' : "0px"})
    
], style={"backgroundColor": "#f8f8f8", 
          "margin-bottom": "10px", 
          "margin-top": "10px",
          'margin-left' : "10px",
          'margin-right' : "10px",
          "color":"#254961",
          "height": "100vh"}) 

@app.callback(
    [Output("count_users", "figure"),
     Output("count_speakers", "figure"),
     Output("сount_titles", "figure"),
     Output("сount_sections", "figure"),
     Output("mean_users_view", "figure"),
     Output("mean_users_titles", "figure"),
     Output("calendar", "figure"),
     Output("titles_top", "figure"),
     Output("speaker_top", "figure"),
     Output("table_titles", "children"),
     Output("table_speakers", "children"),
     Output("section_scatter", "figure"),
     Output("table_section", "children")],
    [Input("events-botton", "n_clicks")], 
    [State("event-checklist", "value"),
     State("date-picker", "start_date"),
     State("date-picker", "end_date")
    ]
)
    
def chart_calendar_events(n, events, start_date, end_date):
    df_users = df1[(df1['event'].isin(events)) & (df1['dt'] >= start_date) & (df1['dt']<=end_date)]
    
    # индикатор подсчета уникальных пользователей
    fig1 = indicator_chart(df_users['user_id'].nunique())
    
    # индикатор подсчета уникальных спикеров
    fig2 = indicator_chart(df_users[df_users['speaker'] != '']['speaker'].str.split(', ').explode().nunique())
    
    # индикатор подсчета выступлений
    fig3 = indicator_chart((df_users[df_users['speaker'] != ''].groupby('time_start', as_index=False)
                  .agg({'title' : lambda x: x.str.split('+').explode().nunique()})['title'].sum()))
    
    # индикатор подсчета секций
    fig4 = indicator_chart((df_users.groupby('section_start', as_index=False)
                  .agg({'section' : 'nunique'})['section'].sum()))
                   
    # индикатор подсчета средней продолжительности просмотра на зрителя
    fig5 = indicator_chart((df_users.groupby('user_id', as_index=False)
                  .agg({'total_time_views_report' : lambda x: (x.sum() / 60)})['total_time_views_report']
                  .mean()
                  .round().astype('int')))
        
    # индикатор подсчета среднего количества просмотренных докладов на зрителя
    fig6 = indicator_chart((df_users[df_users['speaker'] != ''].groupby('user_id', as_index=False)
                  .agg({'report_head' : 'sum'})['report_head']
                  .mean()
                  .round().astype('int')))
                   
    # календарь
    
    event_calendar = df_users.groupby('dt')['user_id'].nunique().reset_index()
    fig7 = calplot(event_calendar, x="dt", y="user_id", 
                gap=0.3, 
                name="Кол-во зрителей",
                month_lines_width=1.5, 
                month_lines_color="#757575")
    fig7.update_traces(
                showscale = True, 
                selector=dict(type='heatmap'),
                zmax=event_calendar['user_id'].max(),
                zmin=event_calendar['user_id'].min(),
                colorscale=px.colors.sequential.Teal,
                colorbar=dict(thickness=14,  tickfont=dict(size=10, family="Lato"),
                title=dict(text='кол-во<br>зрителей<br> ', 
                            font=dict(size=12, family="Lato"))))


    fig7.update_layout( 
                      autosize=False, margin={"r":0,"t":15,"l":10,"b":45},
                      width= 800,
                      height=400,
                      paper_bgcolor="#f8f8f8", 
                      plot_bgcolor='#f8f8f8',
                      font_color="#757575", font_family='Lato', font_size = 14,
                      yaxis=dict(
                      title_text=event_calendar['dt'].dt.strftime('%Y').min(),  tickfont_family='Lato', tickfont_size= 10, tickfont_color='#757575'), 
                      yaxis2=dict(
                      title_text=event_calendar['dt'].dt.strftime('%Y').max(), tickfont_family='Lato', tickfont_size= 10, tickfont_color='#757575'),
                      xaxis=dict(
                      tickfont_family='Lato', tickfont_size= 10, tickfont_color='#757575'), 
                      xaxis2=dict(title_text="",
                      tickfont_family='Lato', tickfont_size= 10, tickfont_color='#757575'))
    
    # 2- я страница 
    # график с топ-8 докладами
    df_titles = titles_tab(df_users)
    fig8 = horizont_plot2(df_titles.iloc[0:10].sort_values(by='number_views'), "number_views" , "title_1",
               'Топ-10 докладов<br>по количеству просмотров<br>  <br> ' , 400, 620)
    
    # график с топ-8 спикерами
    df_speaker = speakers_tab(df_users)
    fig9 = horizont_plot2(df_speaker.iloc[0:10].sort_values(by='report_head'), "report_head", "speaker",
               'Топ-10 спикеров<br>по количеству просмотров<br>  <br> ' , 500, 620)
    
    # таблица с докладами
    data_title = df_titles[['dt', 'title',  'section', 'count_users', 'number_views', 'rate']]
    data_title['dt'] = data_title['dt'].astype('str')
    dt1 = dash_table.DataTable(
        columns=[
            {"name": ["", "Дата"], "id": "dt"},
            {"name": ["", "Доклад"], "id": "title"},
            {"name": ["", "Секция"], "id": "section"},
            {"name": ["Количество зрителей", "Просмотрело"], "id": "number_views"},
            {"name": ["Количество зрителей", "Всего"], "id": "count_users"},
            {"name": ["Количество зрителей", "%"], "id": "rate"},
                    ], 
         data=data_title.to_dict('records'),
         page_size=10, merge_duplicate_headers=True,
         style_data={
                'width': '100px',
                'maxWidth': '100px',
                'minWidth': '100px', 
                'backgroundColor': '#f8f8f8'},
         style_cell={'height': 'auto',
                    'whiteSpace': 'normal', 'textAlign': 'left', 'fontFamily' : 'Lato', 'fontSize' : 14},
         style_cell_conditional=[
            {
                'if': {'column_id': 'title'},
                'width': '250px', 
                'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato', 
            }, 
            {
                'if': {'column_id': 'section'},
                'width': '200px', 'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato'
           },
           {
                'if': {'column_id': 'dt'},
                'width': '60px', 'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato'
           },
           {
                'if': {'column_id': ['count_users', 'number_views', "rate"]},
                'width': '60px', 'textAlign': 'right', 'fontSize' : 14, 'fontFamily' : 'Courier'
           } ],
        style_table={'overflowX': 'auto', 'backgroundColor': '#f8f8f8'},
        style_header={'color': "#1d3a4d",  'fontSize' : 14, 'fontFamily' : 'Lato', 'fontWeight': 'bold', 
                          'backgroundColor': '#eef8f6'})
    html1 = [html.Br(), html.P('Доклады', style = {'font-size' : 18,
                                        'color': "#1d3a4d",
                                        'font-weight' : 'bold'}), dt1]
    # таблица по спикерам
    data_speaker = df_speaker[['speaker',  'place_of_work', 'title', 'report_head','user_id','rate', 'mean_count']]

    dt2= dash_table.DataTable(
        columns=[
            {"name": ["", "Спикер"], "id": "speaker"},
            {"name": ["", "Место работы"], "id": "place_of_work"},
            {"name": ["", "Количество выступлений"], "id": "title"},
            {"name": ["Количество зрителей", "Просмотрело"], "id": "report_head"},
            {"name": ["Количество зрителей", "Всего"], "id": "user_id"},
            {"name": ["Количество зрителей", "%"], "id": "rate"},
            {"name": ["Количество зрителей", "Ср. кол-во просмотров"], "id": "mean_count"},
        ], 
         data=data_speaker.to_dict('records'),
         page_size=10, merge_duplicate_headers=True,
         style_data={
            'width': '100px',
            'maxWidth': '100px',
            'minWidth': '100px', 
            'backgroundColor': '#f8f8f8',
        },
        style_cell={
                'height': 'auto', 'whiteSpace': 'normal', 'textAlign': 'left', 
                'fontFamily' : 'Lato', 'fontSize' : 14
        },
        style_cell_conditional=[
           {
            'if': {'column_id': 'speaker'},
            'width': '200px', 
            'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato', 
           }, 
           {
            'if': {'column_id': 'place_of_work'},
            'width': '200px', 'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato'
           },
         
             {
            'if': {'column_id': ['title', 'report_head', 'user_id', "rate", 'mean_count']},
            'width': '60px', 'textAlign': 'right', 'fontSize' : 14, 'fontFamily' : 'Courier'
           }, 
        ],
        style_table={'overflowX': 'auto', 'backgroundColor': '#f8f8f8'},
        style_header={'color': '#1d3a4d',  'fontSize' : 14, 'fontFamily' : 'Lato', 'fontWeight': 'bold', 
                     'backgroundColor': '#eef8f6'
        })
    
    html2 = [html.Br(), html.P('Спикеры', style = {'font-size' : 18,
                                        'color': "#1d3a4d",
                                        'font-weight' : 'bold'}), dt2]
    # 3 страница 
    # скатерплот по секциям
    df_section = sections_tab(df_users)
    fig10 = scatter_chart(df_section, "mean_depth", "count_users",  "event_new", 'mean_number_views', 950, 600)
    
    # таблица по секциям
    data_section = df_section[['dt',  'section', 'basic', 'count_titles', 'mean_number_views', 'count_users','mean_depth']]
    data_section['dt'] = data_section['dt'].astype('str')
    dt3 = dash_table.DataTable(
        columns=[
            {"name": ["Дата"], "id": "dt"},
            {"name": ["Секция"], "id": "section"},
            {"name": ["Направление"], "id": 'basic'},
            {"name": ["Количество докладов"], "id": "count_titles"},
            {"name": ["Ср. кол-во просмотров"], "id": 'mean_number_views'},
            {"name": ["Количество зрителей"], "id": "count_users"},
            {"name": ["Глубина просмотра, %"], "id": "mean_depth"},
        ], 
         data=data_section.to_dict('records'),
         page_size=10, merge_duplicate_headers=True,
         style_data={
            'width': '100px',
            'maxWidth': '100px',
            'minWidth': '100px', 
            'backgroundColor': '#f8f8f8'},
        style_cell={'height': 'auto','whiteSpace': 'normal', 'textAlign': 'left', 
                    'fontFamily' : 'Lato', 'fontSize' : 14},
        style_cell_conditional=[
           {
            'if': {'column_id': ['section', 'basic']},
            'width': '200px', 
            'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato', 
           }, 
           {
            'if': {'column_id': 'dt'},
            'width': '60px', 'textAlign': 'left', 'fontSize' : 14, 'fontFamily' : 'Lato'
           },
         
           {
            'if': {'column_id': ["count_titles", 'mean_number_views', "count_users", 'mean_depth']},
            'width': '60px', 'textAlign': 'right', 'fontSize' : 14, 'fontFamily' : 'Courier'
           }, 
        ],
        style_table={'overflowX': 'auto', 'backgroundColor': '#f8f8f8'},
        style_header={'color': '#1d3a4d',  'fontSize' : 14, 'fontFamily' : 'Lato', 'fontWeight': 'bold', 
                     'backgroundColor': '#eef8f6'} )
    html3 = [html.Br(), html.P('Секции', style = {'font-size' : 18,
                                        'color': "#1d3a4d",
                                        'font-weight' : 'bold'}), dt3]
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9, html1, html2, fig10, html3
 

if __name__ == '__main__':
    app.run(jupyter_mode="external", port=2009)
    
    

![](https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%94%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4_1.png?raw=true)

![](https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%94%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4_2.png?raw=true)

![](https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%94%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4_3.png?raw=true)

![](https://github.com/KristinaChu/picture/blob/main/oncoforum/%D0%94%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4_4.png?raw=true)

## Список зрителей для рекомендаций 
<a id="recommendation"></a>

Составим список зрителей для каждой секции, в зависимости от глубины её просмотра.

In [ ]:
# сгруппируем данные и выведем глубину просмотра по секции для каждого пользователя
users_section_viewing_depth = (df1
                                .groupby(['user_id', 'section', 'section_start'], as_index=False)
                                .agg(viewing_depth = ('ratio_duration_section', 'first'))
                                 )
# разобьем глубину просмотра на категории 0-10, 10-30, 30-50, 50-70, 70-90, 90-100
cut_labels = ['0-10', '10-30', '30-50', '50-70', '70-90', '90-100']
cut_bins = [-0.1, 9.99, 29.99, 49.99, 69.99, 89.99, 100]

users_section_viewing_depth['groups_viewing_depth'] = pd.cut(users_section_viewing_depth['viewing_depth'],
                                                             bins=cut_bins,
                                                             labels=cut_labels)
users_section_viewing_depth.head()

In [ ]:
# сделаем сводную таблицу выведем для каждой секции и категории по глубине просмотра список зрителей
recommendation_section = (users_section_viewing_depth
                                        .pivot_table(index='section', 
                                                     columns='groups_viewing_depth',
                                         aggfunc=({'user_id': (lambda x: ', '.join(sorted(pd.Series.unique(x))))}))
                                        .reset_index()
                                                )
# скорректируем названия столбцов    
recommendation_section.columns = ['_'.join(col).strip() for col in recommendation_section.columns.values]
recommendation_section.columns = [col.split('_')[-1] if 'user' in col else col.split('_')[0]
                                                for col in recommendation_section.columns.values]
recommendation_section.head()

## Вывод
<a id="finish"></a>

В ходе исследования данных о просмотрах 2-х онкофорумов и 6-ти вебинаров за 2022 и 2023 года удалось выяснить следующее: 

- посмотрело данные мероприятия 3140 уникальных зрителя, из них 2860 смотрели именно онкофорумы
- больше всего зрителей из Санкт-Петербурга(1067) и Москвы(528), а по профессии больше всего врачей хирургов-онкологов (525 зрителей)
- количество докладов, секций и спикеров на онкофоруме в 2023 году, было больше, чем годом ранее, несмотря на то, что сам онкофорум длился на 1 день меньше.
- на онкофоруме в 2023 году зрители в среднем просмотрели больше докладов и провели больше времени. Среднее кол-во просмотренных выступлений зрителем в день на "Мероприятие 12" составило почти 10 докладов, в 2022 году почти на 2 доклада меньше
- распределение кол-ва зрителей и их вовлеченность отличается в зависимости от дня проведения, на выходных днях зрителей в разы меньше, но вовлеченность их не ниже, чем в другие дни.
- в среднем на одно выступление (доклад) приходится 77 зрителей, а посмотрело половину и более доклада(просмотр)- 53 зрителя, на онкофоруме в 2022, в среднем просмотров на 1 доклад было 56, в 2023 году - 51. 
- в среднем кол-во докладов в секции составила 5.6, а вот зрители смотрели в среднем в секциях немного меньше 2 докладов, средняя глубина просмотра длительности секции составила  32% , для онкофорумов данные сильно не отличаются друг от друга
- самыми популярными докладами среди всех докладов стали доклады 1-го дня "Мероприятие 12", так как в этот день был один поток и не было разделения зрителей  
- самые популярные тематики у зрителей всех профессий стали мультидисциплинарное, лекарственная терапия и хирургия, а также оргздав,  зрители смотрят не только секции по своей направленности, но и смежные секции,например, из 529 врачей онкологов-хирургов  63% посмотрели хотя бы один доклад в тематике по хирургии, а доклады с мультидисциплинарной темой, посмотрело больше 69%. Хирургическая секция также популярна у врачей-химиотерапевтов, 60% из них посмотрели хотя бы один доклад по хирургии, но большая доля врачей-химиотерапевтов смотрело мультидисциплинарные  и лекарственные секции

**Рекомендации:**
- выявить и устранить технические причины, приводящие к ошибкам данным по сбору данных о сессиях зрителей(пересекающие по времени секции и др.)
- изменить процедуру сбора информации о профессиях и специализациях пользователей (сделать этот пункт обязательным при регистрации, сделать выбор по конкретному выпадающему списку), это позволит улучшить построение рекомендательных систем
- изменить способ хранения данных о расписании и тематиках секций
- учитывая популярность у всех зрителей таких тематик, как мультидисциплинарное, лекарственная терапия и хирургия, а также оргздав стоит развивать данные секции на онкофорумах
- количество параллельно идущих по времени секций влияет на глубину просмотра и количество зрителей в секциях, стоит учитывать данный факт при составлении расписания


